In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Markdown as md
import pandas as pd

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

time_zone = 'Asia/Jakarta'

In [2]:
import data_source
from datetime import datetime, timedelta
import yfinance as yf

In [3]:
return_column_name = 'return'

In [4]:
def generate_returns_of_market(start_time: datetime, end_time: datetime) -> pd.DataFrame:
    market_ticker = data_source.ticker_market
    market = yf.Ticker(ticker=market_ticker)

    result = market.history(start=start_time, end=end_time)
    returns_of_market = result['Close'].pct_change()[1:].to_frame()

    first_column_name = returns_of_market.columns[0]
    returns_of_market.rename(columns={first_column_name: return_column_name}, inplace=True)

    return returns_of_market

In [5]:
def generate_stock_prices(start_time: datetime, end_time: datetime, tickers: pd.DataFrame, is_more_related: bool) -> pd.DataFrame:
    classified_tickers_list = tickers[tickers['is_more_related'] == is_more_related]['ticker'].to_list()

    tickers_prices = yf.download(tickers=classified_tickers_list, start=start_time, end=end_time)
    tickers_close_prices = tickers_prices['Close'].dropna(axis=1)

    return tickers_close_prices

In [6]:
def calculate_tickers_return(tickers_stock_prices: pd.DataFrame) -> pd.DataFrame:
    tickers_return = tickers_stock_prices.pct_change()[1:]
    return tickers_return

In [7]:
date_column_name = 'date'
ticker_column_name = 'ticker'
return_column_name = 'return'
observation_param_name_for_return = 'param1_r'

In [8]:
def init_observation_df(input_df: pd.DataFrame) -> pd.DataFrame:
  observation_list = []
  for period, row in input_df.iterrows():
    period = period.tz_localize(time_zone)
    for ticker, daily_return in row.items():
      observation_list.append({
          date_column_name: period,
          ticker_column_name: ticker,
          return_column_name: daily_return
      })
  observation_raw_df = pd.DataFrame(observation_list)

  # raw pivot
  observation_df = observation_raw_df.pivot_table(index=date_column_name, columns=ticker_column_name)

  # fixed_pivot
  observation_df.rename(columns={return_column_name: observation_param_name_for_return}, inplace=True)
  observation_df = observation_df.swaplevel(i=0, j=1, axis=1)

  return observation_df

In [9]:
observation_param_name_for_u = 'param2_u'

In [10]:
def calculate_u(observation_df: pd.DataFrame, returns_of_market: pd.DataFrame) -> pd.DataFrame:
  for column, _ in observation_df.items():
    _ticker, _ = column
    observation_df[_ticker, observation_param_name_for_u] = observation_df[_ticker, observation_param_name_for_return] \
      - returns_of_market[return_column_name]
  observation_df = observation_df.reindex(sorted(observation_df.columns), axis=1)
  return observation_df

In [11]:
observation_param_name_for_cu = 'param3_cu'

In [13]:
def calculate_cu(observation_df: pd.DataFrame) -> pd.DataFrame:
  for column, _ in observation_df.items():
    _ticker, _ = column
    observation_df[_ticker, observation_param_name_for_cu] = 0
    cu = 0
    for i, _ in observation_df[_ticker, observation_param_name_for_u].items():
      cu += observation_df.copy().at[i, (_ticker, observation_param_name_for_u)]
      observation_df.at[i, (_ticker, observation_param_name_for_cu)] = cu
  observation_df = observation_df.copy().reindex(sorted(observation_df.columns), axis=1)
  return observation_df

In [ ]:
factor = 30
last_cu_column_name = 'last_cu'

In [ ]:
def select_winner_and_loser_tickers(observation_df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    last_formation_day = observation_df.iloc[-1]
    last_formation_day_dict = last_formation_day.to_dict()
    last_formation_day_list = [{ticker_column_name: key[0], last_cu_column_name: value} for key, value in last_formation_day_dict.items() if key[1] == 'param3_cu']
    last_formation_day_df = pd.DataFrame(last_formation_day_list).copy().sort_values(by=last_cu_column_name, ascending=False)
    ticker_winner_list = last_formation_day_df.iloc[:factor][ticker_column_name].to_list()
    ticker_loser_list = last_formation_day_df.iloc[-factor:][ticker_column_name].to_list()
    return observation_df[ticker_winner_list], observation_df[ticker_loser_list]